---
aliases:
- /2025/08/26/SparkSnowflake
date: '2025-08-26'
output-file: 2025-08-26-sparkinSnowflake.html
title: Running Apache Spark in Snowflake
---

# Running Apache Spark in Snowflake

Dataset: https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv

![image-20240507155528488](./images/image-sparksnowflake.png)



In [ ]:
# Set up the env for Java libraries and enable the Spark Connect Mode
import os
import traceback

os.environ['JAVA_HOME'] = os.environ["CONDA_PREFIX"]
os.environ['JAVA_LD_LIBRARY_PATH'] = os.path.join(os.environ["CONDA_PREFIX"], 'lib', 'server')
os.environ["SPARK_LOCAL_HOSTNAME"] = "127.0.0.1"
os.environ["SPARK_CONNECT_MODE_ENABLED"] = "1"

from snowflake import snowpark_connect
from snowflake.snowpark.context import get_active_session


session = get_active_session()
snowpark_connect.start_session(snowpark_session = session)

# Here is your normal pyspark code. You can of course have them in other Python Cells
spark = snowpark_connect.get_session()
df = spark.sql("show schemas").limit(10)
df.show()

In [ ]:
data = [[2021, "test", "Albany", "M", 42]]
columns = ["Year", "First_Name", "County", "Sex", "Count"]

df1 = spark.createDataFrame(data, schema="Year int, First_Name STRING, County STRING, Sex STRING, Count int")
#display(df1) # The display() method is specific to Databricks notebooks and provides a richer visualization.
df1.show() #The show() method is a part of the Apache Spark DataFrame API and provides basic visualization.



In [ ]:
df_csv = spark.read.csv(f"@aicollege.public.setup/row.csv",
    header=True,
    inferSchema=True,
    sep=",")
#display(df_csv)
df_csv.show()


In [ ]:
df_csv.printSchema()
df1.printSchema()

In [ ]:
df_csv = df_csv.withColumnRenamed("First Name", "First_Name")
df_csv.printSchema()

In [ ]:
df = df1.union(df_csv)
df.show()

In [ ]:
df.filter(df["Count"] > 50).show()

In [ ]:
df.where(df["Count"] > 50).show()



In [ ]:
from pyspark.sql.functions import desc
df.select("First_Name", "Count").orderBy(desc("Count")).show()

In [ ]:
subsetDF = df.filter((df["Year"] == 2009) & (df["Count"] > 100) & (df["Sex"] == "F")).select("First_Name", "County", "Count").orderBy(desc("Count"))
subsetDF.show()

In [ ]:
df.write.mode("overwrite").saveAsTable("AICOLLEGE.PUBLIC.MYFIRSTSPARK")

In [ ]:
df.write.format("json").mode("overwrite").save(f"@aicollege.public.setup/myfirstspark")

In [ ]:
#spark.read.format("json").json(f"@aicollege.public.setup/myfirstspark")



In [ ]:
df.selectExpr("Count", "upper(County) as big_name").show()



In [ ]:
from pyspark.sql.functions import expr
df.select("Count", expr("lower(County) as little_name")).show()

In [ ]:
spark.sql(f"SELECT * FROM AICOLLEGE.PUBLIC.MYFIRSTSPARK").show()

